In [1]:
import geojson
import geopandas as gpd
import random

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors

In [2]:
data = pd.read_csv('./classif/20x20/ml_v3/classif20x20week_callout/csv/transport_results.csv')
#gdf = gpd.read_file('./geojsons/milano-grid.geojson')

with open("./geojsons/milano-grid.geojson") as json_file:
    json_data = geojson.load(json_file)


In [3]:
for i in range(0, len(data)):
    if(data.loc[i].true == 0): #negative label
        json_data.features[int(data.loc[i].cellid)-1].properties = {
                "fill": colors.to_hex([ 0.95,0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.75,
                "cell-type": "no transport",
                "cellId": int(data.loc[i].cellid)
            }
    else: #true: positive
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                "fill": colors.to_hex([ 0.98,0.98, 0.98, 0.45 ], keep_alpha = True),
                "fill-opacity": 0.75,
                "stroke-width": 2.5,
                "stroke-opacity": 0.75,
                "cell-type": "transport",
                "cellId": int(data.loc[i].cellid)
            }

In [4]:
with open("./geojsons/6/truth_20co_ml6_transport.geojson", "w") as json_file:
    geojson.dump(json_data, json_file)

In [5]:
with open("./geojsons/milano-grid.geojson") as json_file:
    json_data = geojson.load(json_file) #reset json_data for next dump

In [6]:
for i in range(0, len(data)):
    if(data.loc[i].predict == 0): #negative label: yellow
        if(data.loc[i].true == 0): #negative label, correctly classified
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                #"stroke": colors.to_hex([ 0.95, 0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill": colors.to_hex([ 0.95,0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.95,
                "cell-type": "no transport",
                "cellId": int(data.loc[i].cellid)
            }
        else: #true: positive - incorrect classification
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                "stroke": colors.to_hex([ 0.95,0.95, 0.95, 0.99 ], keep_alpha = True),
                "fill": colors.to_hex([ 0.95,0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 3.5,
                "stroke-opacity": 0.99,
                "cell-type": "transport",
                "cellId": int(data.loc[i].cellid)
            }
    else: #predicted positive: white
        if(data.loc[i].true == 1): #correct
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                #"stroke": colors.to_hex([ 0.95,0.95, 0.95, 0.95 ], keep_alpha = True),
                "fill": colors.to_hex([ 0.95,0.95, 0.95, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.95,
                "cell-type": "transport",
                "cellId": int(data.loc[i].cellid)
            }
        else: #incorrect classification (true: negative)
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                "stroke": colors.to_hex([ 0.95,0.95, 0.0, 0.99 ], keep_alpha = True),
                "fill": colors.to_hex([ 0.95,0.95, 0.95, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 3.5,
                "stroke-opacity": 0.99,
                "cell-type": "no transport",
                "cellId": int(data.loc[i].cellid)
            }

In [7]:
with open("./geojsons/6/predictions_20co_ml6_transport.geojson", "w") as json_file:
    geojson.dump(json_data, json_file)